In [176]:
# %pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

In [177]:
# %pip install -qU langchain-openai

In [ ]:
# %pip install langchain-chroma
# ollama serve
# ollama run thewindmom/llama3-med42-8b
# ollama run llama3.2

In [179]:
import pandas as pd
from openai import OpenAI

In [180]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

## Load mimic data

In [213]:
notes = pd.read_csv('data/cvd_note.csv')

/tmp/ipykernel_1824013/170795652.py:1: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  notes = pd.read_csv('data/cvd_note.csv')


In [256]:
# notes.sample(3)

In [257]:
# notes['category'].unique().tolist()

In [221]:
# pid = [55730, ]
# docs = notes.loc[notes['subject_id']==pid]
docs = notes.loc[notes['category']=='Discharge summary']

In [222]:
docs.shape

(11717, 17)

In [223]:
docs.to_csv('data/cvd_note_dissum.csv')

In [229]:
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [230]:
loader = CSVLoader(file_path='data/cvd_note_sample.csv',
    csv_args={
    # 'delimiter': ',',
    # 'quotechar': '"',
    'fieldnames': ['subject_id', 'text']
}
    )

In [231]:
docs = loader.load()

In [232]:
embeddings = OllamaEmbeddings(model="llama3.2")
llm = OllamaLLM(model="llama3.2")

In [235]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
# vectordb.persist()
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

In [ ]:
# vectordb = Chroma(persist_directory='data/mimic_embs', embedding_function=embeddings)

In [236]:
prompt = hub.pull("rlm/rag-prompt")
# prompt = PromptTemplate.from_template("{context}\n\nQuestion: {question}")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)



In [255]:
# import langchain
# langchain.debug = True

In [254]:
q = "What diagnosis did this patient has?"
retrieved_docs = retriever.invoke(q )

print(len(retrieved_docs))

context = "\n\n".join(doc.page_content for doc in retrieved_docs)

4


In [250]:
prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": context, "question": q}
).to_messages()

example_messages

[chain/start] [prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "context": "Findings were discussed with [**Doctor First Name 4647**] [**Doctor Last Name 4356**] in person at 10:50 a.m. on\n [**2113-4-5**].\n\nFindings were discussed with [**Doctor First Name 4647**] [**Doctor Last Name 4356**] in person at 10:50 a.m. on\n [**2113-4-5**].\n\nIMPRESSION:\n\n1. In comparison to [**2113-4-11**] exam, there is mild interval\nincrease in the\nsize of ventricles. A small amount of blood layering in\noccipital horns of\nthe lateral ventricles persists. There is no new intracranial\nhemorrhage or infarction.\n\n2. Vasospasm involving the basilar artery, middle and anterior\ncerebral\narteries appears resolved from prior exam. However, moderate\nleft posterior cerebral artery narrowing persists.\n\n3. Residual filling of the coiled left PICA aneurysm measures\n2.7 mm.\n\n4. Stable appearance of the right cerebellar AVM.\n\n5. Sinus disease, as detailed above.\n\nThe study and the 

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: What diagnosis did this patient has? \nContext: Findings were discussed with [**Doctor First Name 4647**] [**Doctor Last Name 4356**] in person at 10:50 a.m. on\n [**2113-4-5**].\n\nFindings were discussed with [**Doctor First Name 4647**] [**Doctor Last Name 4356**] in person at 10:50 a.m. on\n [**2113-4-5**].\n\nIMPRESSION:\n\n1. In comparison to [**2113-4-11**] exam, there is mild interval\nincrease in the\nsize of ventricles. A small amount of blood layering in\noccipital horns of\nthe lateral ventricles persists. There is no new intracranial\nhemorrhage or infarction.\n\n2. Vasospasm involving the basilar artery, middle and anterior\ncerebral\narteries appears resolved from prior exam. However, moderate\nleft post

In [237]:
rag_chain.invoke("What diagnosis did this patient has?")

"I don't know the answer to this question as the provided context does not explicitly mention a diagnosis for the patient. However, it mentions an admitting diagnosis of Subarachnoid Hemorrhage and Aneurysm."

In [191]:
rag_chain.invoke("What diagnosis did patient with subject_id of 55730 has?")

"I don't know the specific diagnosis, but according to the findings and report, the patient has SUBARACHNOID HEMORRHAGE; ANEURYSM. The admitting diagnosis is listed as Subarachnoid Hemorrhage with an associated aneurysm."

In [192]:
rag_chain.invoke("What drugs did this patient take?")

"I don't know what specific medications this patient took, as the provided context does not mention any medication information."

In [201]:
rag_chain.invoke("what is the gender of this patient?")

"I don't know the gender of this patient. The context provides information about a medical examination, but it does not specify the patient's gender."

In [193]:
rag_chain.invoke("how old is this patient?")

"I don't know the patient's age from the provided context. The mention of a 62-year-old man with SAH is in a separate section and may not be related to the current examination. There is no direct information about the patient's age during this examination."

In [195]:
rag_chain.invoke("tell me about this patient's medical history?")

"I don't know the patient's medical history beyond what was mentioned about the fever after subarachnoid hemorrhage and suspected pneumonia. There is no comprehensive description of the patient's overall medical history, past illnesses, or previous treatments. Further context would be needed to provide a more detailed answer."

In [158]:
samples = pd.read_csv('data/cvd_note_sample.csv')

In [251]:
# samples.loc[samples['text'].str.contains('SUBARACHNOID HEMORRHAGE')]

In [252]:
# samples.loc[samples['text'].str.contains('62-year-old')]

In [253]:
# samples.iloc[0]['text']

In [263]:
import pickle, json

In [259]:
with open("rag_results.pkl", "rb") as file:
    results = pickle.load(file)

In [269]:
len(results)

3770

In [270]:
dat = {}
for k in results.keys():
    v = results[k]
    sysm = dat.get(v['medication'], {})
    pids = sysm.get(v['symptom'], {})
    pid = (v['PMID'])
    if pid.isdigit():
        pids["PubMed ID: "+str(pid)] = v['evidence'] + f" https://pubmed.ncbi.nlm.nih.gov/{pid}/"
    else:
        pids[pid] = v['evidence']
    sysm[v['symptom']] = pids
    dat[v['medication']] = sysm

In [271]:
with open("rag-data-url.json", "w") as file:
    json.dump(dat, file, indent=4)  # Save JSON to a file